# Keras
Librería para programar redes neuronales de una manera más sencilla que con TensorFlow. Keras se encuentra en una capa de abstracción por encima de TensorFlow.

[Documentación](https://keras.io/guides/)

In [ ]:
# !pip install tensorflow
# !pip install keras

Empezamos importando librerías

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np

Cargamos los datos de mnist. No vamos a tratar imagenes con redes convolucionales (perdemos la estructura espacial 2D). Todos los pixeles se convertirán en un vector de 28x28 features independientes, que serán las entradas del modelo.

In [ ]:
# Cogemos las imágenes de los dígitos asi como el conjunto de train y test
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

Vemos dimensiones del dataset

In [ ]:
'''
60.000 imagenes de 28x28 pixeles
'''
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
X_train[0]

60.000 imágenes de 28x28 pixeles. Vamos a representar una de ellas

In [ ]:
y_train[0]

In [2]:
import matplotlib.pyplot as plt

plt.imshow(X_train[0], cmap=plt.cm.get_cmap('Greys'));

Cada imagen se compone de 28x28 pixeles, y cada pixel representa una escala de grises que va del 0 al 255. Siendo 0 el blanco y 255 negro.

¿Se te ocurre alguna manera de normalizar los datos?

In [ ]:
0/255

In [ ]:
128/255

In [ ]:
255/255

In [ ]:
X_train = X_train.astype("float32")/255
X_test = X_test.astype("float32")/255

In [ ]:
np.mean(X_train)

In [ ]:
'''
Comprobamos la normalización
'''
X_train[0]

In [ ]:
y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

Guardamos datos para validación. Estos datos se usarán durante el entrenamiento. Otra opción es decirle a keras en la etapa de entrenamiento que reserve un X % de los datos para validar.

In [ ]:
X_val = X_train[-10000:]
y_val = y_train[-10000:]

X_train = X_train[:-10000]
y_train = y_train[:-10000]

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

Montamos la arquitectura de la red neuronal. Se va a componer de:
* **Sequential**: API para iniciar la red neuronal. No cuenta como capa.
* **Flatten**: capa de entrada. Necesita un vector unidimensional. Como tenemos imágenes, esta capa aplana las imagenes (2D) en 1D.
* **Dense**: es una hidden layer. Se compondrá de `n` neuronas y de una función de activación que se aplicará a todas las neuronas de la capa.

Recuerda que es un problema de clasificación multiclase (10 clases) y que por tanto la última capa se compondrá de tantas neuronas como clases tengas.

En cuanto a las funciones de activación es recomendable usar relu en las hidden layer, que tarda menos en entrenar, mientras que la ultima (output) suele ser una softmax.

In [ ]:
model = keras.models.Sequential()

# Capa entrada
model.add(keras.layers.Flatten(input_shape=(28, 28)))

# Hidden layer
model.add(keras.layers.Dense(units = 300,
                            activation='relu'))

# Hidden layer
model.add(keras.layers.Dense(units = 100,
                            activation='relu'))

# Capa salida
model.add(keras.layers.Dense(units = 10,
                            activation='softmax'))

In [ ]:
# Otra manera de declarar la red neuronal
capas = [
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(units = 300, activation='relu'),
    keras.layers.Dense(units = 100, activation='relu'),
    keras.layers.Dense(units = 10, activation='softmax')
]

model = keras.models.Sequential(capas)

Podemos ver las capas, y acceder a sus elementos

In [ ]:
print(model.layers[0])

Podemos ver los pesos de las capas sin entrenar, porque los inicializa aleatoriamente. Los bias los inicializa a 0.

In [ ]:
hidden1 = model.layers[1]
weights, biases = hidden1.get_weights()

In [ ]:
len(weights[0])

In [ ]:
weights.size

In [ ]:
biases[:20]

Establecemos la configuración de ejecución... el compile.

In [ ]:
model.compile(
    optimizer = keras.optimizers.SGD(),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()]
)

In [ ]:
# Equivalente
model.compile(
    optimizer = "sgd",
    loss = "sparse_categorical_crossentropy",
    metrics = ["accuracy"]
)

In [ ]:
# Summary
model.summary()

Entrenamos el modelo. Usamos los datos de entrenamiento. El batch_size es la cantidad de muestras que utiliza el SGD, y las epochs son las iteraciones que realiza en el entrenamiento.

In [ ]:
X_train.shape

In [ ]:
50000/128

In [ ]:
history = model.fit(
    X_train,
    y_train,
    batch_size = 128,
    epochs = 50,
    validation_data = (X_val, y_val) # validation_split = 0.1
)

Podemos reentrenar el modelo. No empieza de nuevo, sino que retoma el entrenamiento anterior.

In [ ]:
model.fit(
    X_train,
    y_train,
    batch_size = 64,
    epochs = 10,
    validation_data = (X_val, y_val) # validation_split = 0.1
)

Veamos el histórico del entrenamiento, para poder representarlo posteriormente.

In [ ]:
print(history.params)
print(history.epoch)
history.history

In [ ]:
history.history

In [ ]:
history.history.keys()

In [ ]:
pd.DataFrame(history.history)

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

Si el modelo no ha ido bien, prueba a cambiar el learning rate, cambia de optimizador y después prueba a cambiar capas, neuronas y funciones de activación.

Ya tenemos el modelo entrenado. Probémoslo con test

In [ ]:
results = model.evaluate(X_test, y_test)
results

In [ ]:
# Cogemos el primero
plt.imshow(X_test[0].reshape(28,28), cmap=plt.cm.get_cmap('Greys'));

In [ ]:
X_test[:1]

In [ ]:
predictions = model.predict(X_test[:1]).round(3)
print(predictions.shape)
predictions

In [ ]:
predictions.argmax()

In [ ]:
model.predict(X_test).argmax(axis=1)

In [ ]:
plt.imshow(X_test[2].reshape(28,28), cmap=plt.cm.get_cmap('Greys'));

### Problema de regresión
Veamos un ejemplo de cómo aplicar una red neuronal de TensorFlow a un problema de regresión.

In [1]:
import tensorflow as tf

# Cargar los datos del Boston Housing dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)

In [4]:
# Cargamos datos
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# housing = fetch_california_housing()
# df = pd.DataFrame(housing.data, columns = housing.feature_names)
# df['target'] = housing['target']
# df.head()

In [5]:
pd.DataFrame(X_train).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12
count,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000
mean,3.745111,11.480198,11.104431,0.061881,0.557356,6.267082,69.010644,3.740271,9.440594,405.898515,18.475990,354.783168,12.740817
std,9.240734,23.767711,6.811308,0.241238,0.117293,0.709788,27.940665,2.030215,8.698360,166.374543,2.200382,94.111148,7.254545
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,188.000000,12.600000,0.320000,1.730000
25%,0.081437,0.000000,5.130000,0.000000,0.453000,5.874750,45.475000,2.077100,4.000000,279.000000,17.225000,374.672500,6.890000
50%,0.268880,0.000000,9.690000,0.000000,0.538000,6.198500,78.500000,3.142300,5.000000,330.000000,19.100000,391.250000,11.395000
75%,3.674808,12.500000,18.100000,0.000000,0.631000,6.609000,94.100000,5.118000,24.000000,666.000000,20.200000,396.157500,17.092500
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.725000,100.000000,10.710300,24.000000,711.000000,22.000000,396.900000,37.970000


Divimos en train, test y validation

In [6]:
# X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data,
#                                                               housing.target)

X_train, X_valid, y_train, y_valid = train_test_split(X_train,
                                                      y_train)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [23]:
X_train.shape

(303, 13)

In [24]:
 X_train.shape[1:]

(13,)

Montamos el modelo. Simplemente se compondrá de una hidden layer, a la que le configuramos una capa previa de entrada de 8 neuronas (las features).

Se trata de un modelo de regresión, por lo que la capa de salida es una única neurona.

In [ ]:
11610/32

In [13]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(30, activation = 'relu',
                      input_shape = X_train.shape[1:]),
    tf.keras.layers.Dense(1)
])

model.compile(loss = "mean_absolute_percentage_error",
             optimizer = "sgd")

history = model.fit(X_train,
                   y_train,
                   epochs = 20,
                   validation_data = (X_valid, y_valid))

Epoch 1/20
10/10 [==============================] - 0s 19ms/step - loss: 92.0892 - val_loss: 77.8431
Epoch 2/20
10/10 [==============================] - 0s 8ms/step - loss: 75.6109 - val_loss: 63.3707
Epoch 3/20
10/10 [==============================] - 0s 12ms/step - loss: 59.1038 - val_loss: 45.0307
Epoch 4/20
10/10 [==============================] - 0s 9ms/step - loss: 36.0165 - val_loss: 28.2733
Epoch 5/20
10/10 [==============================] - 0s 9ms/step - loss: 23.0511 - val_loss: 23.2660
Epoch 6/20
10/10 [==============================] - 0s 8ms/step - loss: 18.2786 - val_loss: 20.9545
Epoch 7/20
10/10 [==============================] - 0s 8ms/step - loss: 15.7946 - val_loss: 18.3838
Epoch 8/20
10/10 [==============================] - 0s 9ms/step - loss: 14.2895 - val_loss: 17.7660
Epoch 9/20
10/10 [==============================] - 0s 9ms/step - loss: 13.5001 - val_loss: 16.6508
Epoch 10/20
10/10 [==============================] - 0s 10ms/step - loss: 12.8100 - val_loss: 16.1

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 30)                420       
                                                                 
 dense_3 (Dense)             (None, 1)                 31        
                                                                 
Total params: 451
Trainable params: 451
Non-trainable params: 0
_________________________________________________________________


In [15]:
mse_test = model.evaluate(X_test, y_test)
print(mse_test)

4/4 [==============================] - 0s 5ms/step - loss: 14.8787


14.878665924072266


In [18]:
y_pred = model.predict(X_test[:5])
y_pred

array([[ 5.846137],
       [17.04153 ],
       [22.739452],
       [26.448528],
       [26.120148]], dtype=float32)

### Guardar modelo
Para guardar el modelo, en el formato de Keras (HDF5). 

In [19]:
model.save("my_keras_model.h5")

In [29]:
# Lo volvemos a cargar
model = tf.keras.models.load_model("callback_model.h5")

In [28]:
del model

In [30]:
model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 4ms/step - loss: 13.3583


13.358282089233398

### Callbacks
Son funciones predefinidas de Keras a aplicar durante el entrenamiento
Por ejemplo, `ModelCheckpoint` sirve para que el modelo se vaya guardando tras cada epoch. Así no perdemos el progreso en caso de que decidamos interrumpir el entrenamiento. El callback recibe como argumento el nombre del objeto donde queremos que se guarde el modelo entrenado.

In [27]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("callback_model.h5")
history = model.fit(X_train,
                   y_train,
                   epochs=30,
                   callbacks = [checkpoint_cb])

Epoch 1/30
10/10 [==============================] - 0s 6ms/step - loss: 10.3934
Epoch 2/30
10/10 [==============================] - 0s 6ms/step - loss: 10.2467
Epoch 3/30
10/10 [==============================] - 0s 6ms/step - loss: 10.1358
Epoch 4/30
10/10 [==============================] - 0s 7ms/step - loss: 9.5841
Epoch 5/30
10/10 [==============================] - 0s 8ms/step - loss: 10.1790
Epoch 6/30
10/10 [==============================] - 0s 10ms/step - loss: 9.8055
Epoch 7/30
10/10 [==============================] - 0s 7ms/step - loss: 9.7647
Epoch 8/30
10/10 [==============================] - 0s 7ms/step - loss: 9.7816
Epoch 9/30
10/10 [==============================] - 0s 7ms/step - loss: 9.4840
Epoch 10/30
10/10 [==============================] - 0s 7ms/step - loss: 10.2333
Epoch 11/30
10/10 [==============================] - 0s 7ms/step - loss: 9.7262
Epoch 12/30
10/10 [==============================] - 0s 7ms/step - loss: 9.5622
Epoch 13/30
10/10 [========================

KeyboardInterrupt: 

### Early Stopping
Interrumpe el entrenamiento cuando no ve progreso en el set de validación. Para ello tiene en cuenta un numero de epochs llamado `patience`. Se puede combinar con el callback

In [31]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, )
history = model.fit(X_train,
                   y_train,
                   epochs=2000,
                    validation_data = (X_valid, y_valid),
                   callbacks = [early_stopping_cb])

Epoch 1/2000
10/10 [==============================] - 0s 14ms/step - loss: 9.3253 - val_loss: 14.1833
Epoch 2/2000
10/10 [==============================] - 0s 9ms/step - loss: 9.1761 - val_loss: 12.3861
Epoch 3/2000
10/10 [==============================] - 0s 10ms/step - loss: 9.7355 - val_loss: 15.4178
Epoch 4/2000
10/10 [==============================] - 0s 11ms/step - loss: 9.8083 - val_loss: 12.9600
Epoch 5/2000
10/10 [==============================] - 0s 13ms/step - loss: 9.4095 - val_loss: 12.3090
Epoch 6/2000
10/10 [==============================] - 0s 12ms/step - loss: 9.0876 - val_loss: 13.5602
Epoch 7/2000
10/10 [==============================] - 0s 9ms/step - loss: 8.9169 - val_loss: 12.9708
Epoch 8/2000
10/10 [==============================] - 0s 13ms/step - loss: 9.3044 - val_loss: 12.8442
Epoch 9/2000
10/10 [==============================] - 0s 9ms/step - loss: 9.6792 - val_loss: 14.4303
Epoch 10/2000
10/10 [==============================] - 0s 9ms/step - loss: 9.2458 - v